In [ ]:
import pandas as pd
import os
import ast
import numpy as np
import pdb

import google.generativeai as genai  # ✅ Only this import is needed for Gemini

genai.configure(api_key="")  # Replace with your actual API key

model = genai.GenerativeModel('gemini-2.0-flash')  # or 'gemini-1.5-flash'

instruction = "Think of you as a tourist guide at USA and you know many attractive places at california"
question = "What are the best places to be visited in california?"

full_prompt = instruction + "\n\n" +question

response = model.generate_content(full_prompt)

print(response.text)